# Machine learning tutorial - Exercise

Machine learning (ML) has seen rapid growth in the past decade and has revolutionized many fields in both industry and research, achieving state-of-the-art results and solving highly complex problems. Due to the sheer vastness of this topic, it is impossible to give a full overview in a single tutorial.

Thus, we are focussing on equipping you with all the basic tools necessary to start into the field of ML and be able to utilize its power for your own research project. In particular, we will use one of the most popular ML frameworks, [pytorch](https://pytorch.org/), to solve two typical tasks: classification and regression.

## Choosing a dataset

There exist several curated "benchmark" datasets that have been used in many papers and competitions for comparing the performance of different machine learning algorithms. Since we assume the audience to be interested in scientific datasets in particular, we chose the [Higgs dataset](https://archive.ics.uci.edu/ml/datasets/HIGGS) for our classification task. For the regression task, we chose the [life expectancy dataset](https://www.kaggle.com/datasets/kumarajarshi/life-expectancy-who) by the World Health Organisation (WHO). Let's get started!

**Tip: There are several interesting websites to look for openly available benchmark datasets, like [google dataset search](https://datasetsearch.research.google.com/), [kaggle](https://www.kaggle.com/datasets/) or [paperswithcode](https://paperswithcode.com/datasets). There are also built-in datasets that you can easily access using the pytorch API, see [this Link](https://pytorch.org/vision/stable/datasets.html) for more information or [this Link](https://www.tensorflow.org/datasets/catalog/overview) for the `TensorFlow` equivalent.** 

## Downloading the Higgs dataset

Since the original files of the Higgs dataset are huge, we use a random subset of 25% of the original dataset for this tutorial. For convenience, we prepared two H5 files that contain the low-level kinematic inputs of the particles and higher level features that are computed from the lower level ones, respectively. 

In [ ]:
%%shell
wget -O higgs_highlevels.h5 https://wolke.physnet.uni-hamburg.de/index.php/s/oJDBfnkiQZZYYcq/download
wget -O higgs_lowlevels.h5 https://wolke.physnet.uni-hamburg.de/index.php/s/j7sJGjrMY8faaYb/download

## The Higgs dataset: Exploration

Before we start with machine learning, it is always important to do some exploratory data analysis and test if it coincides with our domain-knowledge expectations. Also, we should always look for unwanted effects such as missing data or outliers.

A useful library for data analysis is [pandas](https://pandas.pydata.org/), which stores data in a so called "data frame", which is essentially a table, where each row represents a sample of our dataset and each column represents a specific feature of this sample. For example, in case of our Higgs dataset, each row represents a single particle collision event and each column represents a specific feature of this event, like the momentum in the transverse direction ($p_{\mathrm{T}}$) of a particle jet, or the mass of a two-jet system (also referred to as "dijet") in the event.

The pandas library also comes with built-in plotting functions and allows for efficient exploration, preprocessing and manipulation of our data.

In [ ]:
# Import necessary libraries
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import torch
from torch import nn
import time

In [ ]:
# load h5 file into pandas dataframe, we will look at the high level variables
# at first

df_highlvl = pd.read_hdf("higgs_highlevels.h5")

Let's take a look at the features contained in the dataset

In [ ]:
df_highlvl.columns

We see there are several mass and energy variables, as well as kinematic variables of leptons. There is also a `class_label` column, which tells us if an event comes from a Higgs decay ("signal", label=1) or from a known Standard Model particle decay ("background", label=0). In Machine Learning, these truth-level variables are often referred to as *label* or *target* values.

Let's plot the distributions of these variables for the signal and background events separately:

In [ ]:
# create mask for signal/background distribution comparison
sig_mask = (df_highlvl['class_label'] == 1)

# drop first column (class_label) and rearrange other columns 
# to a 6x2 matrix (as numpy array) to use as "mosaic" for
# the subplot_mosaic call
colname_matrix = df_highlvl.columns.to_numpy()[1:].reshape(6,2)
fig, axs = plt.subplots(6,2, figsize=(14, 21))

for row in range(6):
    for col in range(2):
        colname_tmp = colname_matrix[row, col]

        # set current axis to subplot index
        plt.sca(axs[row, col])
        
        # define bin edges and use histogram plotting function
        # of pandas dataframe
        bin_edges = np.linspace(df_highlvl[colname_tmp].min(),
                                df_highlvl[colname_tmp].max(), 100)
        df_highlvl[~sig_mask][colname_tmp].plot.hist(bins=bin_edges,
                                                     label="background")

        df_highlvl[sig_mask][colname_tmp].plot.hist(histtype="step", bins=bin_edges,
                                            label="signal")

        plt.xlabel(colname_tmp)

        # the phi angular coordinate is typically flat, which is
        # why it's the only feature we would like to see not in log scale
        if not "phi" in colname_tmp:
            plt.yscale("log")

        plt.legend(loc="upper right")

plt.show()
plt.close()

We can see what is often the case in particle physics: Signal and background distributions are - at least by eye - very similar. Neural networks try to find patterns in the data and utilize the subtle changes between signal and background to learn a function that maps from the features we see here to a single variable that has a much higher discriminative power.

The strength of Deep Neural Networks (DNNs) is that they can learn nonlinear functions over the full input space and thus can also pick up on correlations in higher dimensions, which we can not see easily by just looking at the one dimensional marginal distributions of the features.

## Preprocessing the data

We have now looked at the distributions and while we could not see any "strange" patterns or outliers in our data, we still need to make sure that there are not too many infinite or NaN values. Before we implement our ML model and train it, we also need to make sure we have all the datasets we need for a statistically sound analysis and that the features are represented in a way that is beneficial for the network to learn.

At first, we investigate `INF` and `NaN` values.

In [ ]:
# There are is a function called isna() that we can call on pandas dataframes
# to check for NaN values. For INFs we could use isin([-np.inf, np.inf]), but
# it is easier to use the numpy function isfinite() to check for both at the
# same time.

# We sum over the boolean array which is 1 at all occurrences that are
# Nan or INF per column to count them
(~np.isfinite(df_highlvl)).sum()

As we can see, we got lucky this time: There are no INF or NaN values in this dataset. This is often the case in particle physics data analysis, since usually the input data comes in through a sophisticated data preprocessing pipeline that is curated by your research collaboration, but it might be different in other fields.

Now comes another important step: We need to split our dataset into three parts: The **training** set, the **validation** set and the **test** set. These datasets need to be **completely separate** from each other, in order to guarantee statistically independent results. Let's quickly discuss why this is necessary:

The training set is used for training the DNN. A DNN has many parameters or weights, that are adjusted during the training to best minimize a certain loss function - in our case of binary classification, this will be the [binary crossentropy](https://en.wikipedia.org/wiki/Cross_entropy) loss. So in the end, we just fit a very complex function to our input dataset, which comes with a caveat: At first, the DNN will learn useful patterns in the data, but at some point the network will learn the "noise", i.e. the particular statistical fluctuations in our signal and background distributions, to further decrease the loss. So the DNN just "memorizes" the data instead of learning the general pattern that helps solve our statistical problem.

What we are interested in, however, is learning a general function that does not only perform well on the training set, but also generalizes well to other, previously unseen datasets or samples. This is why we need a second, statistically independend dataset when we evaluate our model. This is the **validation** set. With the validation set, we can give an estimate on the **generalization performance** of our trained model. The validation set is also useful to detect **overfitting**, which is another word describing the above mentioned phenomenon of the DNN model starting to learn the noise of the training set.

In practice, we regularly evaluate our model both on the training and on the validation set during training. At first, the loss is minimized for both sets, but at some point the validation loss stalls out at a certain value or even increases, while the training loss goes further down. At this point, we can stop the training and select the model with the minimum validation loss, since it will yield the optimal generalization performance. This procedure is also referred to as **model selection**.

However, also the model selection introduces a bias, since we select the model with the best generalization performance *on that particular validation set*. This is why we need a third statistically independent dataset, the **test set** to finally have an objective performance measure that is affected by neither the training nor the model selection procedure.

There is no general rule how to optimally split up your dataset, but typically the majority of data will go into the training set, while test and validation set are kept roughly the same. We will choose a 70/15/15 percent split for our first test.

In [ ]:
# At first, we will move the target column to a separate variable and then drop
# this column from the original dataframe

y_full = df_highlvl['class_label']
x_full = df_highlvl.drop(columns=['class_label'])

In [ ]:
# import train_test_split function from scikit_learn, then first split off
# training set, and then split remainder into validation and test sets

from sklearn.model_selection import train_test_split

# split off training set (70% of samples)
x_train, x_remain, y_train, y_remain = train_test_split(x_full, y_full,
                                                        train_size=0.7,
                                                        random_state=42)


# split remainder into test and validation sets
# (15% each, corresponding to half of the remaining non-training samples)
x_val, x_test, y_val, y_test = train_test_split(x_remain, y_remain,
                                                train_size=0.5,
                                                random_state=42)


In [ ]:
# let's see if the numbers check out
n_full = x_full.shape[0]
train_percent = (x_train.shape[0]/n_full)*100
val_percent = (x_val.shape[0]/n_full)*100
test_percent = (x_test.shape[0]/n_full)*100

print(f"Train set corresponds to {train_percent:.2f}% of the full data.")
print(f"Validation set corresponds to {val_percent:.2f}% of the full data.")
print(f"Test set corresponds to {test_percent:.2f}% of the full data.")

It seems everything checks out! Now we also would like to apply some preprocessing to our data. Depending on your project, preprocessing - or "data wrangling" - can take up a significant proportion of your work. In the Higgs dataset example we only apply a "standard scaling", which means that we subtract the mean and divide by the standard deviation in each feature, such that all features end up with a mean of 0 and a standard deviation of 1.

This makes sure our input features are all in a similar range, which can help algorithms based on gradient descent - which our DNN optimization is - have a faster and better convergence behaviour; (For more information read e.g. [this blog post](https://www.analyticsvidhya.com/blog/2020/04/feature-scaling-machine-learning-normalization-standardization/) for an in-depth discussion of feature scaling an when to use which scaling method).

In [ ]:
# import standard scaler from scikit-learn
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# fit (i.e. do the mean/std computation based on the training set features)
# and transform training data

x_train = scaler.fit_transform(x_train)

# important: the standard scaling is always with respect to the training set
# values of mean and std! So DO NOT call fit_trainsform again on the other sets
# but just transform using the already fitted scaler.

x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

In [ ]:
# check if it worked
print("Means of training set:\n", x_train.mean(axis=0), "\n")
print("Standard deviations of training set:\n", x_train.std(axis=0), "\n\n")

print("Means of validation set:\n", x_val.mean(axis=0), "\n")
print("Standard deviations of validation set:\n", x_val.std(axis=0), "\n\n")

print("Means of test set:\n", x_test.mean(axis=0), "\n")
print("Standard deviations of test set:\n", x_test.std(axis=0), "\n\n")

It seems it worked! While means and standard deviations are 0 and 1 respectively for the training set, we get slightly higher/lower values in the validation and test sets, which reflects the fact that we used the standard scaler based on the training set values only and then just applied the transformations to the other two sets.

We are now good to go! We loaded our dataset, checked for outliers and `INF`/`NaN` values and then preprocessed it. Now we will build our neural network using `pytorch`!

## Building the `pytorch` model

Writing custom models in `pytorch` is relatively straightforward. For this tutorial, we will set up what is often referred to as a "dense" or "fully connected" neural network, which is essentially a model that consists of several layers where each layer contains a certain number of nodes and all nodes from one layer are connected to all the nodes in the adjacent layers.

These fully connected networks (FCNs) have an input layer and an output layer and so called "hidden layers" that are in between. The number of nodes in the input layer corresponds to the number of input features of our dataset. The output layer usually has one node for a binary classification problem, which outputs the probability of an event belonging to the "positive" class. However, there are also multi-class and multi-label classification problems, where multiple output nodes are necessary.

The number of hidden layers and of nodes inside them is set by the user, since there is no "optimal" rule how many one should take. For one ML problem, fewer layers with many nodes are beneficial, while for other problems it might be the opposite. Varying and finding the setting of number of layers and nodes per layer that yields the best result for a given problem is often referred to as ["hyperparameter optimization"](https://en.wikipedia.org/wiki/Hyperparameter_optimization) and it is a task that needs to be re-done for each problem or network architecture. One can use several methods to tune hyperparameters, ranging from "brute force" methods such as [grid search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) or random search to more sophisticated algorithms based on Bayesian optimization. 

Common layers of neural networks are provided in the [`torch.nn`](https://pytorch.org/docs/stable/nn.html) module of `pytorch`. A fully connected layer can be implemented by using [`torch.nn.linear`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear) and providing the number of input and output values of this layer. This means that, instead of providing the concrete number of nodes in the current layer, you also need to think about how many nodes the next layer should have. Let's take an example: Assume we had 5 input features and would like to create a layer for that. Our next layer, i.e. the first hidden layer, should have 16 nodes, which is why we would need to create the layer like this: `layer = nn.Linear(5, 16)`. Note that now the next linear layer in your model needs to have the same number of inputs as the output of the previous layer, so `16` in this case. You can now stack many of these layers like this until the output layer, which should have a number of output values of `1` for our binary classification problem.

However, this is not the full story yet! In order for the FCN to be able to learn complex non-linear functions of the data, we need to apply an [activation function](https://en.wikipedia.org/wiki/Activation_function) to the outputs of our layers. The most common and useful activations are also contained in the `torch.nn` module and we can simply add them to our model. One common activation function is the ["Rectified Linear Unit"](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)) (ReLU), which is zero at negative values values of $x$ and $x$ for values $x\ge0$. We can add this activation by calling `torch.nn.ReLU()` after a linear layer. 

`ReLU` activations are especially useful in input and hidden layers. For the output layer, however, we need to choose a different activation function: For a classification problem, we want the network output to reflect the probability of a sample to belong to a certain class. This means that we need to make sure that our output nodes produce a number ranging from 0 to 1 and that the sum over all probabilities equals 1. For a multi-class classification problem, we can achieve this using [`torch.nn.softmax`](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html#torch.nn.Softmax). For a binary classification problem, we are only interested in the probability of belonging to the "positive" class, which we can achieve using a single node with a [`torch.nn.Sigmoid`](https://pytorch.org/docs/stable/generated/torch.nn.Sigmoid.html#torch.nn.Sigmoid) activation.

To put it all together, we can create a fully connected model in `pytorch` by stacking linear layers, followed by an activation (such as ReLU) and making sure that the activation of our output layer fits our problem at hand (e.g. sigmoid activation for binary classification). In practice, we implement this appending layers and activations to a python `list` and once we are done, we create a model by providing the list to a [`torch.nn.Squential`](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) object, which is just a sequential container that builds our final model from our provided modules.

In the code below, we have conveniently wrapped this into a separate class called `Classifier` which contains other useful methods that come in handy later in this tutorial.

In [ ]:
class Classifier(nn.Module):
    def __init__(self, layers, n_inputs=5):
        super().__init__()

        # the layers variable will be the list that contains the individual
        # linear layers and their activations
        self.layers = []
        for nodes in layers:
            # in the first iteration, n_inputs will correspond to the number
            # of input features in our dataset. After the input layer, the
            # value of n_inputs is set to the number of output features, such
            # that the layers are always stacked in the correct way
            self.layers.append(nn.Linear(n_inputs, nodes))
            self.layers.append(nn.ReLU())
            n_inputs = nodes

        # the final linear layer with a Sigmoid activation for binary
        # classification
        self.layers.append(nn.Linear(n_inputs, 1))
        self.layers.append(nn.Sigmoid())
        
        # build pytorch model as sequence of our layers
        self.model_stack = nn.Sequential(*self.layers)

    def forward(self, x):
        # the forward call just takes data (x) and sends it through the model
        # to produce an output (in our case a number between 0 and 1)
        return self.model_stack(x)

    def predict(self, x):
        # the predict method sets the model to evaluation mode and only then
        # computes the model prediction of given data (x). For convenience,
        # we already make sure that the output prediction is a numpy array, so
        # we can use it easier in the final evaluation of the model.
        with torch.no_grad():
            self.eval()
            x = torch.tensor(x)
            prediction = self.forward(x).detach().cpu().numpy()
        return prediction

## Running the training

For running the training, we need to implement a "training loop". In Machine Learning, we typically optimize our model in an iterative way. This means that we use our training data multiple times to adjust the weights of the FCN and get better performance. Additionally, we often face the problem that our dataset is too large to fit into the memory at once. This is why usually we need to run the training in a "batched" fashion.

For, this, we use a tool called a [`DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader) that is included in `pytorch`. The `DataLoader` takes our dataset and randomly samples small batches of data from our full dataset. We then loop over all the batches and for each batch result adjust the weights of the network, until we looped over all the samples in our dataset this way. This is the "inner training loop" and once it concludes, we say that one "epoch" of training is finished.

Also in the inner training loop, we need to use an [`optimizer`](https://pytorch.org/docs/stable/optim.html), which takes care of adjusting the model parameters to minimize the loss at each training step. There exist many optimization algorithms that one can use to approach the minimum of the loss function. Two very famous algorithms are [Stochastic Gradient Descent (SGD)](https://en.wikipedia.org/wiki/Stochastic_gradient_descent) that is implemented in `pytorch` as [`torch.optim.SGD`](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html#torch.optim.SGD) or [ADAM](https://optimization.cbe.cornell.edu/index.php?title=Adam), which is implemented as [`torch.optim.Adam`](https://pytorch.org/docs/stable/generated/torch.optim.Adam.html#torch.optim.Adam).

The inner training loop needs to follow a fixed structure in `pytorch` that looks like this:

```python

# Send (batch of) inputs through the model
model_outputs = model(inputs)

# compute the loss of the model outputs using the model predictions and truth labels (targets)
loss = loss_function(model_outputs, targets)

# make sure gradients are reset to zero in each iteration
optimizer.zero_grad()

# backpropagate the loss. This will compute the gradient of the loss
# w.r.t. the weights of our model, which we need to update the weights
loss.backward()

# finally, actually update the weights
optimizer.step()
```

This "inner loop" is a loop over all the batches in our data, and it ends once we used up all of them.

Then there is the "outer training loop", which simply repeats the inner loop for a user-specified number of multiple epochs, but it consists of two parts:

1. The training part: Here, we just run the model through the inner training loop as described and also keep track of the **average** loss per batch.
2. The validation part: We set the model to **evaluation mode** (turn off gradients computation, set specific layers to freeze trained values) and then send our validation set through it. Again we compute the loss (also often in a batched fashion) and in the end compute the average loss per batch.

With these two parts, we can plot a **loss curve** of our training: The x axis shows the epoch and the y axis is the loss value. We can draw two curves, one for the training and one for the validation loss. This allows us to detect overfitting (train loss decreases, validation loss stalls out or increases) and also observe the convergence behaviour of our training.

This was a lot of theory, let's put this into practice!

In [ ]:
# In order to be able to use the pytorch dataloader, we need to create pytorch
# datasets from our training, validation and test sets (and their respective
# target vectors)

from torch.utils.data import TensorDataset, DataLoader
from torch import optim
import torch.nn.functional as F

train_set = TensorDataset(torch.tensor(x_train),
                          torch.from_numpy(y_train.to_numpy()).reshape(-1, 1))
val_set = TensorDataset(torch.tensor(x_val),
                        torch.from_numpy(y_val.to_numpy()).reshape(-1, 1))

# create DataLoader objects
train_loader = DataLoader(train_set, batch_size=1024, shuffle=True)
val_loader = DataLoader(val_set, batch_size=1024)

# set number of epochs you want to train
epochs = 5

# build model using a single layer with 64 neurons
clsf_model = Classifier(layers=[64], n_inputs = x_train.shape[1])

# Use Adam optimizer to keep track of model parameter updates.
# The "lr" parameter is the learning rate and describes the "step size"
# that we take at each weight update in the direction of the largest
# negative gradient value. It is again a hyperparameter we need to tune,
# but we leave it at the default of 1e-3 for now.
optimizer = optim.Adam(clsf_model.parameters(), lr=1e-3)

# Define loss function. For a binary problem, use binary crossentropy loss.
loss = F.binary_cross_entropy


In [ ]:
# outer training loop

# define empty lists for storage of training and validation losses
train_losses = []
val_losses = []

start = time.time()

for epoch in range(epochs):
    
    running_train_loss = 0
    
    # make sure model is in training mode
    clsf_model.train()

    # training part of outer loop = inner loop
    for batch in train_loader:
        
        data, targets = batch
        output = clsf_model(data)
        tmp_loss = loss(output, targets)
        optimizer.zero_grad()
        tmp_loss.backward()
        optimizer.step()
        
        running_train_loss += tmp_loss.item()
    
    print(f"Train loss after epoch {epoch+1}: {running_train_loss/len(train_loader)}")
    train_losses.append(running_train_loss/len(train_loader))
    
    ## validation part of outer loop
    
    running_val_loss = 0
    # deactivate gradient computation
    with torch.no_grad():
        
        # set model to evaluation mode
        clsf_model.eval()
        
        # loop over validation DataLoader
        for batch in val_loader:
            
            data, targets = batch
            output = clsf_model(data)
            tmp_loss = loss(output, targets)
            running_val_loss += tmp_loss.item()
        
        mean_val_loss = running_val_loss/len(val_loader)
        print(f"Validation loss after epoch {epoch+1}: {mean_val_loss}")
        
        # If the validation loss of the model is lower than that of all the
        # previous epochs, save the model state
        if epoch == 0:
            torch.save(clsf_model.state_dict(), "./min_val_loss_model.pt")
        elif (epoch > 0) and (mean_val_loss < np.min(val_losses)):
            print("Lower loss!")
            torch.save(clsf_model.state_dict(), "./min_val_loss_model.pt")
        
        val_losses.append(mean_val_loss)

end = time.time()
print(f"Done training {epochs} epochs!")
print(f"Training took {end-start:.2f} seconds!")

## Check training convergence

Our first training is done! Let's check if it converged nicely: We saved all the training and validation losses in a `list` which we can now plot!

In [ ]:
plt.plot(np.arange(epochs), train_losses, label="training")
plt.plot(np.arange(epochs), val_losses, label="validation")
plt.ylabel("loss")
plt.xlabel("epochs")
plt.legend(loc="upper right")
plt.show()
plt.close()

It seems the training converged! However, we see that both training and validation losses still go down, so we could have trained a little longer here probably.

We can now do the final step of our analysis and evaluate our model!

## Evaluate the trained model

The evaluation is done using the **test set**. We send it through the model to get the predictions, and compare them to our truth labels to see how accurate it is. There are several metrics one can use to quantify the model performance. The simplest is the accuracy: the fraction of correctly predicted samples divided by all samples in the test set.

Another more sophisticated but very common method is to plot a [`Receiver Operator Characteristic (ROC) curve`](https://en.wikipedia.org/wiki/Receiver_operating_characteristic). The advantage of this approach is that it takes into account both correct predictions of the positive class as well as wrongly classified negative class samples and it does so for different thresholds on the classifier output.

A typical approach is to plot this curve and quote its integral, the "Area Under Curve" (AUC) value as a single performance metric. The AUC is also often quoted in machine learning research papers to compare the performance of different state-of-the-art model architectures with each other.

In [ ]:
# import ROC/AUC computation from scikit-learn package
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score

# make sure we take the model with the lowest validation loss
clsf_model.load_state_dict(torch.load("./min_val_loss_model.pt"))

y_score = clsf_model.predict(x_test)

test_acc = accuracy_score(y_test, np.around(y_score))
print(f"Model accuracy: {test_acc:.2f}")

fpr, tpr, thresholds = roc_curve(y_test, y_score)

test_auc = roc_auc_score(y_test, y_score)
print(f"Model AUC: {test_auc:.2f}")



In [ ]:
# plot ROC curve
plt.plot(fpr, tpr)
plt.plot(np.linspace(0,1,100), np.linspace(0,1,100), color="grey",
         linestyle="dashed")

plt.xlabel("False Positive Rate (FPR)")
plt.ylabel("True Positive Rate (TPR)")
plt.show()
plt.close()

Great! We did a full analysis run, starting with the raw data, using some preprocessing, then training a binary classifier using `pytorch` and evaluating it.

The accuracy is 71% and the AUC is 0.79. The dashed line in the ROC curve is simply the diagonal and stands for the worst possible (i.e. random) performance, which corresponds to an AUC of 0.5. Our result is quite higher than that, but still, we might do better. Now it's your turn!

## Task: Improve the result!

Try to improve the AUC and also try to understand how varying different parameters of the training affect the result.

Some starting points might be:

- So far, we only considered a single hidden layer with 64 nodes. Maybe you can improve the model architecture a bit?
- We didn't play with other hyperparameters, like:
    - the learning rate
    - batch size
    - the optimizer
    - activations for the hidden layers
- We also didn't use the low-level features that are in the `higgs_lowlevels.h5`. Using the same preprocessing procedure as before, maybe you can integrate (some of) those and see if they actually improve our performance.

Play around a bit and get to know the `pytorch` library better! Feel free to copy code from anything we used so far!

## Regression task

The second machine learning task for this tutorial is a linear regression: we try to predict a single variable from multiple input features. For this, we use the World Health Organisation (WHO) [life expectancy dataset](https://www.kaggle.com/datasets/kumarajarshi/life-expectancy-who) where we try to predict the life expectancy of a country given different medical, economical and social factors.

In [ ]:
%%shell
wget -O life_expectancy_data.csv https://wolke.physnet.uni-hamburg.de/index.php/s/Bdnmoit483Xrnmo/download

In [ ]:
# load data from csv file
df_who = pd.read_csv("life_expectancy_data.csv")

# take a look at dataframe
df_who

As we can see, this dataset isn't as convenient as the Higgs one. We still have different features, but not all of them seem to be numeric. For example, we have the name of the country as one column and also a country's "status", which can be, for example, a "developing" country. We can use the `dtypes` method on the `DataFrame` object to check for types.

In [ ]:
df_who.dtypes

We see that we are lucky: Most columns have a numeric type, like `int` or `float`. `String` columns typically have the `object` type in pandas dataframes. We see that there are two string-encoded columns: the `Country` column, which simply contains the name of the country and the `Status` column, which tells us about the development status of a country.

For a machine learning model to work on this data, we would need to turn these two columns into a numerical representation. This means we need to employ some kind of **encoding**. This is a topic for itself, but in principle there exist two major ways how we can encode strings that are categorical (i.e. there exist a limited number of unique string values, such as the country names):

1. Ordinal encoding: If there is an intrinsic order to the strings, we can simply convert the string values into integers, ranging from 1 to the number of unique string values. For example, say our column was "shirt size" and we had the values "S", "M", "L", "XL" and "XXL", then we could just encode them with integers from 1 to 5. This is possible because the size strings carry meaning: "S" is a smaller size than "XL", which is why it makes sense to assign it a smaller value.

2. One-hot encoding: If there is no intrinsic order to the strings (for example in case of our country names), we can do what is called a [one-hot encoding](https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/). In a one-hot encoding, we **create a new column for each unique string value in the original column** and this column is 1 where this unique value occurs and 0 everywhere else. For example, let's imagine our country names would only contain the names of three countries: France, Germany and Belgium. To one-hot encode this column, we would create three new columns: one column is 1 where the string "France" occurs in the original column and else it is 0. The second column is 1 where the string "Germany" occurs and else it is again 0. And the same for the "Belgium" string. This can of course inflate our dataset quite a bit, in particular if we have many unique values.

Let's investigate the two `object` columns a bit. We can use the `value_counts` method to get the unique values in a column and how often they occur.

In [ ]:
df_who["Country"].value_counts()

We see that there are 193 countries in that column. Some occur 16 times, which means they contain individual rows for each of the years 2000 to 2015, other only occur once (probably due to missing data in several years). If we were to one-hot encode this column, we would add 193 values, which seems a bit much. For now we decide to postpone the decision what to do with this column and just drop it for the first test.

In [ ]:
# drop Country column for now
df_who = df_who.drop(columns=["Country"])

Now let's take a look at the `Status` column:

In [ ]:
df_who["Status"].value_counts()

The `Status` column only has two unique values: `Developed` or `Developing`. For this column, one could argue to use an ordinal encoding, since developed and developing countries have certain unique quantifiable differences in terms of economics and also in other areas. This could lead us to use a higher integer value for developed countries and a lower one for still developing ones. However, since there are only two unique values in this column, we go for a one-hot encoding here.

In [ ]:
# simple pandas function to one-hot encode a column: get_dummies

encoded_status = pd.get_dummies(df_who["Status"], prefix="status")

encoded_status

We see that there are now two new columns: one for the developed and one for developing. We could actually just drop one of these columns, since they are perfectly correlated, so the whole information is already contained only in one of them. Note that this would result in a single column that is 0 for one value and 1 for the other one, which could also be interpreted as an ordinal encoding of the original feature. So if there are only two features, a one-hot encoding already has kind of an ordinal structure to it.

In [ ]:
# drop one of the new columns
encoded_status = encoded_status.drop(columns=["status_Developing"])

# drop the original status columns from the WHO DataFrame
df_who = df_who.drop(columns=["Status"])

# add the new status_Developing column to the WHO DataFrame
df_who["status_Developed"] = encoded_status["status_Developed"]


Let's again check the datatypes in our `DataFrame`. We should see that it doesn't contain `object` dtypes any more.

In [ ]:
df_who.dtypes

Now we need to check again for `INF` and `NaN` values. 

In [ ]:
(~np.isfinite(df_who)).sum()

This time we are not as as lucky as in the Higgs dataset. We have several columns where there are multiple `NaN` or `INF` values. Lets check particularly for `INF` values.

In [ ]:
np.isinf(df_who).sum()

We see that there are no `INF` values, which means all the non-finite values we saw in the previous test are `NaN`.

There are several ways how we can mitigate `NaN` values: If there are only few rows affected, we can simply drop these. However, this is usually not ideal since we also remove information from the data. If there exist columns where a majority of rows is `NaN`, we can also consider dropping the column, since it doesn't carry much information that we can use.

Often, a better choice than simply dropping the values is **imputing** the values, i.e. replace the missing values with some other value. Typical choices for the replacement value are the mean or the median of the feature distribution. However, also these methods are not really great, since they do not preserve the relationships among variables. The optimal way would be to use some kind of domain knowledge to impute the values. For example, if you were an expert in the population growth behavior of developing countries and you knew that their population growth is best described with an exponential function, we could simply take the existing `Population` values of a country, do an exponential fit and set the missing values accordingly.

Beyond that, there exist many other methods to impute missing values. You can read more on the topic for example [here](https://odsc.medium.com/data-imputation-beyond-mean-median-and-mode-6c798f3212e3). Another approach is based on the k-nearest neighbors algorithm: Here we impute the mean only of the $k$ neighboring data points, computed as the euclidean distance across all features. You can read more about this topic [here](https://machinelearningmastery.com/knn-imputation-for-missing-values-in-machine-learning/). We will use this algorithm here, since we can easily use the `sklearn` imputer called `KNNImputer`. 

In [ ]:
from sklearn.impute import KNNImputer

# use imputer defaults for now
imputer = KNNImputer()

# transform the dataframe and save it into a new DataFrame object
df_who_clean = pd.DataFrame(imputer.fit_transform(df_who), columns=df_who.columns)


Let's check for `NaN` values again:

In [ ]:
(~np.isfinite(df_who_clean)).sum()

It worked! We imputed all the `NaN` values using the kNN imputation approach! Now we are done with the first data cleaning step. Now we split the data again into training, validation and test set and scale the features to have zero mean and unit standard deviation.

Before we do that, we also split off the life expectany column from the `DataFrame`, since this is the "target" variable that we want to predict from all the other features.

In [ ]:
# Split the cleaned DataFrame into targets and features
targets = df_who_clean['Life expectancy']
data = df_who_clean.drop(columns=['Life expectancy'])

# Split into training/validation/test sets according to 70/15/15 split
from sklearn.model_selection import train_test_split

# split off training set (70% of samples)
x_train, x_remain, y_train, y_remain = train_test_split(data, targets,
                                                        train_size=0.7,
                                                        random_state=42)


# split remainder into test and validation sets
# (15% each, corresponding to half of the remaining non-training samples)
x_val, x_test, y_val, y_test = train_test_split(x_remain, y_remain,
                                                train_size=0.5,
                                                random_state=42)

In [ ]:
# scale features to zero mean and unit standard deviation
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Note: We also make cast all values to 32 bit floats here, since this is what
# pytorch models expect as inputs by default, while numpy arrays are stored
# in 64 bit precision
x_train = scaler.fit_transform(x_train).astype("float32")
x_val = scaler.transform(x_val).astype("float32")
x_test = scaler.transform(x_test).astype("float32")
y_train = y_train.astype("float32")
y_val = y_val.astype("float32")
y_test = y_test.astype("float32")

Now we are almost good to go! We now need to build the `pytorch` model. The good news is: we can use the exact same network architecture as before! The only difference is the output activation and the loss function we use.

In a classification task, we want a number between 0 and 1 for our output node, which we can interpret as the probability of a sample belonging to the positive class. In this case, the loss we want to minimize is the binary cross-entropy. Maybe you know that a random variable with only two possible outcomes (famous example: throwing a coin) follows a [bernoulli distribution](https://en.wikipedia.org/wiki/Bernoulli_distribution) and the binary cross-entropy is simply the negative log likelihood of its probability mass function.

The case for a regression task is a bit different: We would like to predict a continuous variable and the prediction should be as close as possible to the samples' true value of that variable. Since there is no constraint on the range of the output, we can simply take a **linear** activation function in the end. Since we already implement the fully connected layers as linear layers in `pytorch`, this means we just leave out the activation after the output layer. The loss function to minimize in a regression task is typically the [mean squared error](https://pytorch.org/docs/stable/generated/torch.nn.functional.mse_loss.html), since we simply want to minimize the difference between the predicted and true value.

Let's start building our regression model:

In [ ]:
# The exact same code as for the Classifier we used before, just with the final
# output activation removed
class Regressor(nn.Module):
    def __init__(self, layers, n_inputs=5):
        super().__init__()

        self.layers = []
        for nodes in layers:
            self.layers.append(nn.Linear(n_inputs, nodes))
            self.layers.append(nn.ReLU())
            n_inputs = nodes

        self.layers.append(nn.Linear(n_inputs, 1))
        # here we just removed the Sigmoid activation

        self.model_stack = nn.Sequential(*self.layers)

    def forward(self, x):
        return self.model_stack(x)

    def predict(self, x):
        with torch.no_grad():
            self.eval()
            x = torch.tensor(x)
            prediction = self.forward(x).detach().cpu().numpy()
        return prediction

Next, we build the actual model, optimizer and data loaders that we will use in the training

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
from torch import optim
import torch.nn.functional as F

train_set = TensorDataset(torch.tensor(x_train),
                          torch.from_numpy(y_train.to_numpy()).reshape(-1, 1))
val_set = TensorDataset(torch.tensor(x_val),
                        torch.from_numpy(y_val.to_numpy()).reshape(-1, 1))

# create DataLoader objects
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32)

# set number of epochs you want to train. Because our dataset is very small,
# we can start with a large number of epochs
epochs = 100

# build model using a single layer with 64 neurons
reg_model = Regressor(layers=[64], n_inputs = x_train.shape[1])

# Use Adam optimizer again
optimizer = optim.Adam(reg_model.parameters(), lr=1e-3)

# Use mean squared error loss
loss = F.mse_loss

Run training loop

In [ ]:
# define empty lists for storage of training and validation losses
train_losses = []
val_losses = []

start = time.time()

# outer training loop
for epoch in range(epochs):
    
    running_train_loss = 0
    
    # make sure model is in training mode
    reg_model.train()

    # training part of outer loop = inner loop
    for batch in train_loader:
        
        data, targets = batch
        output = reg_model(data)
        tmp_loss = loss(output, targets)
        optimizer.zero_grad()
        tmp_loss.backward()
        optimizer.step()
        
        running_train_loss += tmp_loss.item()
    
    print(f"Train loss after epoch {epoch+1}: {running_train_loss/len(train_loader)}")
    train_losses.append(running_train_loss/len(train_loader))
    
    ## validation part of outer loop
    
    running_val_loss = 0
    # deactivate gradient computation
    with torch.no_grad():
        
        # set model to evaluation mode
        reg_model.eval()
        
        # loop over validation DataLoader
        for batch in val_loader:
            
            data, targets = batch
            output = reg_model(data)
            tmp_loss = loss(output, targets)
            running_val_loss += tmp_loss.item()
        
        mean_val_loss = running_val_loss/len(val_loader)
        print(f"Validation loss after epoch {epoch+1}: {mean_val_loss}")
        
        # If the validation loss of the model is lower than that of all the
        # previous epochs, save the model state
        if epoch == 0:
            torch.save(reg_model.state_dict(), "./min_val_loss_reg_model.pt")
        elif (epoch > 0) and (mean_val_loss < np.min(val_losses)):
            print("Lower loss!")
            torch.save(reg_model.state_dict(), "./min_val_loss_reg_model.pt")
        
        val_losses.append(mean_val_loss)

end = time.time()
print(f"Done training {epochs} epochs!")
print(f"Training took {end-start:.2f} seconds!")

## Evaluate the training

Again, we take a look at the convergence of the training by plotting the loss curves:

In [ ]:
plt.plot(np.arange(epochs), train_losses, label="training")
plt.plot(np.arange(epochs), val_losses, label="validation")
plt.ylabel("loss")
plt.xlabel("epochs")
plt.legend(loc="upper right")
plt.show()
plt.close()

The convergence of the training looks good! Now we need to compute some performance measures, such as mean and median squared and absolue errors, the [R2 score](https://en.wikipedia.org/wiki/Coefficient_of_determination) and the [explained variance score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.explained_variance_score.html)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error, max_error, r2_score, explained_variance_score

reg_model.load_state_dict(torch.load("./min_val_loss_reg_model.pt"))

y_pred = reg_model.predict(torch.tensor(x_test))

print("Classification performance report")

print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred):.2f}")
print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_pred):.2f}")
print(f"Median Absolute Error: {median_absolute_error(y_test, y_pred):.2f}")
print(f"Max Error: {max_error(y_test, y_pred):.2f}")
print(f"R2 score: {r2_score(y_test, y_pred):.2f}")
print(f"Explained Variance Score: {explained_variance_score(y_test, y_pred):.2f}")

That's it! We now also finished the regression task using a `pytorch` model! Next, it's your turn again!

## Task: Improve the regression model

Similar to the classification task, take some time to explore the hyperparameters of the analysis. Here are some leads:

- In the preprocessing, we used imputation for the missing values. Maybe with a different imputation method to handle `NaN` values, we can improve the performance? One hyperparameter for the kNN imputation is the value of $k$ that is used (default is 5). Maybe this wasn't the optimal value to use here.
- Again we didn't tune any of the hyperparameters here (numbers of nodes/layers, learning rate, activation functions, batch size etc.), this would be a good starting point to improve the baseline performance probably

